In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#ratings = {'false': 0, 'true': 1, 'mixture': 4, 'outdated': 6, 'miscaptioned': 7, 'scam': 10}
ratings = {'false': 0, 'true': 1, 'mixture': 2, 'miscaptioned': 3, 'scam': 4}

In [3]:
def page_content(weblinks, articles, key):
    pagelinks = []
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append(url.get('href'))
    for page in pagelinks:
        html = BeautifulSoup(requests.get(page).content, 'html.parser')
        content = html.find_all('article')[0]
        content = content.find_all('html')
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, ratings[key]))

In [4]:
articles = []

In [5]:
for key in ratings.keys():
    for i in range(1,80):
        if i==1:
            page = requests.get("https://www.snopes.com/fact-check/rating/" + key)
        else:
            page = requests.get("https://www.snopes.com/fact-check/rating/" + key + "/" + str(i))
        soup = BeautifulSoup(page.content, 'html.parser')
        weblinks = soup.find_all('article')
        page_content(weblinks, articles, key)

In [6]:
print(len(articles))

4375


In [7]:
#more miscaptioned articles from snopes
for i in range(1,52):
    if i==1:
        page = requests.get("https://www.snopes.com/video/")
    else:
        page = requests.get("https://www.snopes.com/video/" + str(i) + "/")
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('article')
    page_content(weblinks, articles, 'miscaptioned')

In [9]:
len(articles)
import pandas as pd
pd.DataFrame(articles, columns = ['Text', 'Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
4982,"[In November 2018, after the Camp Fire broke o...",3
4983,[As President Trump and his administration wer...,3
4984,"[On 7 November 2018, a brief article published...",3
4985,"[In early October 2018, First Lady Melania Tru...",3


In [10]:
count = [0]*5
for i in articles:
    count[i[1]] = count[i[1]] + 1
count

[875, 875, 875, 1487, 875]

In [11]:
#more mixture articles from politifact
for i in range(1,37):
    pagelinks = []
    if i==1:
        page  = requests.get('https://www.politifact.com/factchecks/list/?category=truth-o-meter&ruling=half-true')
    else:
        page = requests.get('https://www.politifact.com/factchecks/list/?page=' + str(i) + '&category=truth-o-meter&ruling=half-true')
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('div', {'class':"m-statement__quote"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.politifact.com' + url.get('href'))
    for page in pagelinks:
        html = BeautifulSoup(requests.get(page).content, 'html.parser')
        content = html.find_all('article', {'class':"m-textblock"})[0]
        content = content.find_all('html')
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, 2))

In [12]:
pd.DataFrame(articles, columns = ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
6032,[If you take away his religious-conservative v...,2
6033,"[Standing on a stage in Iowa, amid all his com...",2
6034,"[For months, the antitax group Club for Growth...",2
6035,[Under attack from Alan Keyes during a GOP deb...,2


In [15]:
for a in articles:
    if len(a[0]) == 2:
        articles.remove(a)

In [16]:
len(articles)

5646

In [17]:
count = [0]*5
for i in articles:
    count[i[1]] = count[i[1]] + 1
count

[797, 797, 1846, 1409, 797]

In [22]:
temp = pd.DataFrame(articles, columns = ['Text', 'Label'])
temp.to_csv('news-temp.csv')

In [33]:
import urllib.request

In [ ]:
from translate import Translator
translator = Translator(to_lang = 'en')
translated = translator.translate('Comment allez-vous')
translated

In [ ]:
#false articles about coronavirus from poynter
for i in range(1, 100):
    if i==1:
        url = ('https://www.poynter.org/ifcn-covid-19-misinformation/?covid_countries=0&covid_rating=51174&covid_fact_checkers=47485')
    else:
        url = ('https://www.poynter.org/ifcn-covid-19-misinformation/page/' + str(i) + '/?covid_countries=0&covid_rating=51174&covid_fact_checkers=47485#038;covid_rating=51174&covid_fact_checkers=47485')
    r = urllib.request.Request(url, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    page = urllib.request.urlopen(r)
    soup = BeautifulSoup(page, 'html.parser')
    weblinks = soup.find_all('h2', {'class':"entry-title"})
    pagelinks = []
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append(url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        full_article = html.find_all('a',{'class':"button entry-content__button entry-content__button--smaller"})[0]
        full_article_url = full_article.get('href')
        r = urllib.request.Request(full_article_url, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('div',{'class':"story-article__content__element--text"})
        print(content)
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')

In [101]:
#checking
url = ('https://www.poynter.org/ifcn-covid-19-misinformation/?covid_countries=0&covid_rating=51174&covid_fact_checkers=47485')
r = urllib.request.Request(url, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
page = urllib.request.urlopen(r)
soup = BeautifulSoup(page, 'html.parser')
weblinks = soup.find_all('h2', {'class':"entry-title"})
pagelinks = []
for link in weblinks:
    url = link.find_all('a')[0]
    pagelinks.append(url.get('href'))
for page in pagelinks:
    r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    news = urllib.request.urlopen(r)
    html = BeautifulSoup(news, 'html.parser')
    full_article = html.find_all('a',{'class':"button entry-content__button entry-content__button--smaller"})[0]
    full_article_url = full_article.get('href')
    r = urllib.request.Request(full_article_url, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    news = urllib.request.urlopen(r)
    html = BeautifulSoup(news, 'html.parser')
    content = html.find_all('div')
    print(content)
    clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
    text = clean_content.replace('\n','')
    text = text.replace('\xa0', '')

[<div id="div-gpt-ad-1589720950581-0" style="width: 1px; height: 1px;">
<script>
        googletag.cmd.push(function() { googletag.display('div-gpt-ad-1589720950581-0'); });
      </script>
</div>, <div id="div-gpt-ad-1589721120803-0" style="width: 1px; height: 1px;">
<script>
        googletag.cmd.push(function() { googletag.display('div-gpt-ad-1589721120803-0'); });
      </script>
</div>, <div id="container"></div>, <div id="div-gpt-ad-1588347376785-0">
<script>
        googletag.cmd.push(function() { googletag.display('div-gpt-ad-1588347376785-0'); });
      </script>
</div>, <div id="div-gpt-ad-1508230408514-0" style="display:none;">
<script>
        googletag.cmd.push(function() { googletag.display('div-gpt-ad-1508230408514-0'); });
      </script>
</div>]
[<div id="div-gpt-ad-1589720950581-0" style="width: 1px; height: 1px;">
<script>
        googletag.cmd.push(function() { googletag.display('div-gpt-ad-1589720950581-0'); });
      </script>
</div>, <div id="div-gpt-ad-158972112

KeyboardInterrupt: 

In [91]:
#false coronavirus articles from politifact
for i in range(1,8):
    pagelinks = []
    if i==1:
        page  = requests.get('https://www.politifact.com/factchecks/list/?category=coronavirus&ruling=false')
    else:
        page = requests.get('https://www.politifact.com/factchecks/list/?page=' + str(i) + '&category=coronavirus&ruling=false')
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('div', {'class':"m-statement__quote"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.politifact.com' + url.get('href'))
    for page in pagelinks:
        html = BeautifulSoup(requests.get(page).content, 'html.parser')
        content = html.find_all('article', {'class':"m-textblock"})[0]
        content = content.find_all('html')
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text,0))

In [94]:
#false coronavirus articles from politifact
for i in range(1,4):
    pagelinks = []
    if i==1:
        page  = requests.get('https://www.politifact.com/factchecks/list/?category=coronavirus&ruling=pants-fire')
    else:
        page = requests.get('https://www.politifact.com/factchecks/list/?page=' + str(i) + '&category=coronavirus&ruling=pants-fire')
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('div', {'class':"m-statement__quote"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.politifact.com' + url.get('href'))
    for page in pagelinks:
        html = BeautifulSoup(requests.get(page).content, 'html.parser')
        content = html.find_all('article', {'class':"m-textblock"})[0]
        content = content.find_all('html')
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text,0))

In [ ]:
import pandas as pd
data = pd.DataFrame(articles, columns= ['Text', 'Label'])
data.to_csv('news_temp.csv')

In [ ]:
len(articles)

In [ ]:
count = [0]*5
for i in articles:
    count[i[1]] = count[i[1]] + 1
count

In [39]:
#true articles about coronavirus from livemint
for i in range(1, 100):
    pagelinks = []
    if i==1:
        page = requests.get('https://www.livemint.com/topic/coronavirus')
    else:
        page = requests.get('https://www.livemint.com/topic/coronavirus/page-' + str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2', {'class':"headline"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.livemint.com' + url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('article')[0]
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, 1))

In [40]:
pd.DataFrame(articles, columns= ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
7621,Nurses check the temperatures of visitors as p...,1
7622,The diagnosis marks the first positive case of...,1
7623,Only those who have proper permission of the o...,1
7624,ICMR officials are in talks with at least 60-7...,1


In [ ]:
page = requests.get('https://www.livemint.com/topic/coronavirus')
soup = BeautifulSoup(page.content, 'html.parser')
weblinks = soup.find_all('h2', {'class':"headline"})
for link in weblinks:
    url = link.find_all('a')[0]
    pagelinks.append('https://www.livemint.com' + url.get('href'))
for page in pagelinks:
    r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    news = urllib.request.urlopen(r)
    html = BeautifulSoup(news, 'html.parser')
    content = html.find_all('article')[0]
    clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
    text = clean_content.replace('\n','')
    text = text.replace('\xa0', '')
    print(text)

In [55]:
#true articles about coronavirus from livemint
for i in range(101, 200):
    pagelinks = []
    if i==1:
        page = requests.get('https://www.livemint.com/topic/coronavirus')
    else:
        page = requests.get('https://www.livemint.com/topic/coronavirus/page-' + str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2', {'class':"headline"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.livemint.com' + url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('article')[0]
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, 1))

In [56]:
pd.DataFrame(articles, columns = ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
7899,The coronavirus outbreak has disrupted flights...,1
7900,Odisha has not reported any positive case so f...,1
7901,In the hypothetical worst-case scenarios for e...,1
7902,'All candidates who are scheduled to come onsi...,1


In [160]:
count = [0]*5
for i in articles:
    count[i[1]] = count[i[1]] + 1
count

[2712, 3059, 1846, 1409, 1349]

In [ ]:
data = pd.DataFrame(articles, columns= ['Text', 'Label'])
data.to_csv('news_temp.csv')

In [59]:
#more articles on scams from livemint
searches = ['https://www.livemint.com/Search/Link/Keyword/phish', 'https://www.livemint.com/Search/Link/Keyword/fraud', 'https://www.livemint.com/Search/Link/Keyword/scam']
for search in searches:
    page = requests.get(search)
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2', {'class':"headline"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.livemint.com' + url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('article')[0]
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, 4))

In [60]:
pd.DataFrame(articles, columns = ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
7993,The meeting will be chaired by the health mini...,4
7994,These cyber attacks were aimed at heightening ...,4
7995,A film producer said he recently ordered liquo...,4
7996,Alexa is a voice-controlled virtual assistant ...,4


In [113]:
count = [0]*5
for i in articles:
    count[i[1]] = count[i[1]] + 1
count

[2712, 3059, 1846, 1409, 887]

In [62]:
#true articles about coronavirus from livemint
for i in range(201, 300):
    pagelinks = []
    if i==1:
        page = requests.get('https://www.livemint.com/topic/coronavirus')
    else:
        page = requests.get('https://www.livemint.com/topic/coronavirus/page-' + str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2', {'class':"headline"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.livemint.com' + url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('article')[0]
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, 1))

In [162]:
count = [0]*5
for i in articles:
    count[i[1]] = count[i[1]] + 1
count

[2712, 3059, 1846, 1409, 1349]

In [63]:
pd.DataFrame(articles, columns = ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
7993,The meeting will be chaired by the health mini...,4
7994,These cyber attacks were aimed at heightening ...,4
7995,A film producer said he recently ordered liquo...,4
7996,Alexa is a voice-controlled virtual assistant ...,4


In [71]:
#fake coronavirus articles from boom
for i in range(1,25):
    if i==1:
        page = requests.get('https://www.boomlive.in/search?search=coronavirus%20fake%20news')
    else:
        page = requests.get('https://www.boomlive.in/search?search=coronavirus%20fake%20news&search_type=all&page='+str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2',{'class':"entry-title"})
    pagelinks = []
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.boomlive.in' + url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('div',{'class':"story"})
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text,0))

In [102]:
#fake news from politifact
for i in range(1,30):
    pagelinks = []
    if i==1:
        page  = requests.get('https://www.politifact.com/factchecks/list/?category=truth-o-meter&ruling=false')
    else:
        page = requests.get('https://www.politifact.com/factchecks/list/?page=' + str(i) + '&category=truth-o-meter&ruling=false')
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('div', {'class':"m-statement__quote"})
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.politifact.com' + url.get('href'))
    for page in pagelinks:
        html = BeautifulSoup(requests.get(page).content, 'html.parser')
        content = html.find_all('article', {'class':"m-textblock"})[0]
        content = content.find_all('html')
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text, 0))

In [103]:
pd.DataFrame(articles, columns = ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
9916,[The newest tool in climate skeptics' arsenal:...,0
9917,"[On his Dec. 8, 2009, radio show, conservative...",0
9918,[After the New Orleans Saints squeaked by with...,0
9919,[Fox News Channel host Glenn Beck recently fou...,0


In [88]:
for i in range(20,30):
    if i==1:
        page = requests.get('https://www.boomlive.in/fake-news')
    else:
        page = requests.get('https://www.boomlive.in/fake-news/'+str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2',{'class':"entry-title"})
    pagelinks = []
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append('https://www.boomlive.in' + url.get('href'))
    for page in pagelinks:
        r = urllib.request.Request(page, headers= {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        news = urllib.request.urlopen(r)
        html = BeautifulSoup(news, 'html.parser')
        content = html.find_all('div',{'class':"story"})
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text,0))

In [89]:
pd.DataFrame(articles, columns=['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
8791,[A nearly one year old video from Uttar Prades...,0
8792,[Viral social media posts claiming Shadab Faro...,0
8793,[A set of eight images showing various seized ...,0
8794,[1. Video Shows Chinese Policemen Killing Coro...,0


In [157]:
#articles on scams from ftc
for i in range(0,30):
    if i==0:
        page = requests.get('https://www.consumer.ftc.gov/features/scam-alerts')
    else:
        page= requests.get('https://www.consumer.ftc.gov/features/scam-alerts?page='+str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    weblinks = soup.find_all('h2')[7:-3]
    pagelinks = []
    for link in weblinks:
        url = link.find_all('a')[0]
        pagelinks.append(url.get('href'))
    for page in pagelinks:
        html = BeautifulSoup(requests.get(page).content, 'html.parser')
        content = html.find_all('div', {'class':"content"})[0]
        clean_content = BeautifulSoup(str(content), 'html.parser').get_text()
        text = clean_content.replace('\n','')
        text = text.replace('\xa0', '')
        articles.append((text,4))

In [168]:
len(articles)

10375

In [166]:
for a in articles:
    if len(a[0]) == 2:
        articles.remove(a)

In [167]:
news = []
for a in articles:
    news.append((a[0].replace('\t',''),a[1]))

In [169]:
pd.DataFrame(news, columns = ['Text','Label'])

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
10370,"When you book a hotel room online, you expect ...",4
10371,It’s tough enough to find a job or start your ...,4
10372,The Federal Trade Commission cracked down on a...,4
10373,"""There is currently money available NOW right ...",4


In [170]:
data = pd.DataFrame(news, columns = ['Text','Label'])
data

,Text,Label
0,[During anti-police-brutality protests that sw...,0
1,"[In June 2020, a rumor started to circulate on...",0
2,[Rumors are surging in the wake of George Floy...,0
3,"[On June 4, 2020, a security fence was erected...",0
4,"[In early June 2020, social media users shared...",0
...,...,...
10370,"When you book a hotel room online, you expect ...",4
10371,It’s tough enough to find a job or start your ...,4
10372,The Federal Trade Commission cracked down on a...,4
10373,"""There is currently money available NOW right ...",4


In [171]:
data.to_csv('Snopes-data.csv', index = False)